In [23]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (
    ensemble,
    preprocessing,
    tree,
)

from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
)

from yellowbrick.classifier import (
    ConfusionMatrix,
    ROCAUC,
)

from yellowbrick.model_selection import (
    LearningCurve,
)

from pathlib import Path

In [24]:
ROOT_FILE = Path.cwd().parent
DATA_FILE = ROOT_FILE / "datasets" / "titanic3.csv"

Yellow Brick é uma biblioteca de visualização e diagnóstico para aprendizado de máquina que visa capacitar os usuários a visualizar os processos internos de seus modelos de aprendizado de máquina e permitir que eles tomem decisões mais informadas durante o processo de modelagem, facilitando a compreensão das forças e fraquezas de seus modelos.

## Faça uma pergunta

Essa corresponde a primeira etapa do processo de machine learning, onde você deve se perguntar o que deseja descobrir com os dados que possui. Para o exemplo do livro, será criado um modelo preditivo para responder uma pergunta: O indivíduo sobreviveu ou não ao naufrágio do Titanic? Terá-se como base os dados de passageiros do navio, como idade, sexo, classe social, etc.

É uma pergunta de classificação, pois estamos fazendo a predição de um rotulo, que é a sobrevivência ou não do passageiro. Se fosse uma pergunta de regressão, seria algo como: Qual a idade do passageiro?

## Colete os dados

A segunda etapa é a coleta dos dados, que pode ser feita de diversas formas, como por exemplo, através de um web crawler, que é um programa que navega pela web e coleta os dados de interesse. Pode ser feita pelo download de um dataset, como o do Kaggle, ou até mesmo através de uma API.

Para o exemplo do livro, estou mostrando duas maneiras, uma através de uma URL e outra através do download de um dataset.

In [25]:
url = ("https://hbiostat.org/data/repo/titanic3.csv")
df = pd.read_csv(url)

## OU

df = pd.read_csv(DATA_FILE)


Tem-se os seguintes dados:

- **pclass**: Classe do passageiro (1 = primeira classe; 2 = segunda classe; 3 = terceira classe)

- **survived**: Sobreviveu ou não (0 = Não; 1 = Sim)

- **name**: Nome do passageiro

- **sex**: Sexo do passageiro

- **age**: Idade do passageiro

- **sibsp**: Número de irmãos e cônjuges a bordo

- **parch**: Número de pais e filhos a bordo

- **ticket**: Número do ticket

- **fare**: Tarifa do passageiro

- **cabin**: Número da cabine

- **embarked**: Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton)

- **boat**: Bote salva-vidas

- **body**: Número de identificação do corpo

- **home.dest**: Destino

## Limpe os dados

A maioria dos modelo do scikit-learn exige que os dados sejam numéricos. Os modelos podem falhar caso recebam valores ausentes. Além disso alguns modelos podem ter melhores desempenhos se os dados estiverem padronizados.

Podem também existir os chamados leaky features(dados que vazam informações). Essas variaveis contém informações sobre o futuro ou o resultado que se deseja prever

In [26]:
df.dtypes # Verificando os tipos de dados

pclass         int64
survived       int64
name          object
sex           object
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object

Os tipos de dados mais comuns são:

- **int64**: Números inteiros

- **float64**: Números reais

- **datetime64[ns]**: Data e hora

- **object**: String, mas pode ser uma combinação entre strings e outros tipos

No geral tipos inteiros não apresentam problemas. Tipos float podem apresentar problemas com valores ausentes. Tipos data e string deverão ser convertidos para números.

Tipos string com baixa cardinalidade (poucos valores únicos) são chamados de colunas de categoria, é possível gerar colunas dummy (binárias) para cada valor único

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


Temos 6 variaveis númericas, 7 variaveis categóricas e 1 variavel booleana

In [32]:
df.shape # Verificando o tamanho do dataset

(1309, 14)

In [34]:
df.describe() # Verificando as estatísticas do dataset

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881138,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413493,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [37]:
df.describe().iloc[:, :2] # iloc é um método para selecionar 
# linhas e colunas por números inteiros. Neste caso, estamos
# selecionando todas as linhas e as duas primeiras colunas

,pclass,survived
count,1309.000000,1309.000000
mean,2.294882,0.381971
std,0.837836,0.486055
min,1.000000,0.000000
25%,2.000000,0.000000
50%,3.000000,0.000000
75%,3.000000,1.000000
max,3.000000,1.000000


- **count:**  quantidade de valores não nulos

- **mean:** média dos valores

- **std:** desvio padrão

- **min:** valor mínimo

- **25%:** primeiro quartil

- **50%:** segundo quartil (mediana)

- **75%:** terceiro quartil

- **max:** valor máximo

Count é interessa para identificar valores ausentes. A mediana é mais robusta que a média, pois não é afetada por outliers.

Min e máx são importantes para identificar outliers. O desvio padrão é uma medida de dispersão, quanto maior o desvio padrão, maior a dispersão dos dados.

```python

df.isnull() # retorna um dataframe com valores booleanos, onde True indica que o valor é nulo

df.isnull().sum() # retorna a soma dos valores nulos de cada coluna

```

In [38]:
df.isnull().sum() # Verificando a quantidade de valores nulos

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

```python

df.isnull().mean() # Verificando a porcentagem de valores nulos

```

Útil para identificar colunas com muitos valores nulos

In [48]:
df.isnull().sum(axis=1).loc[:10] # Verificando a quantidade de valores nulos por linha

0     1
1     1
2     2
3     1
4     2
5     1
6     1
7     2
8     1
9     2
10    1
dtype: int64

A coluna body deve ser removida pois contém muitos valores nulos e pode ser considerada uma leaky feature (informa que o passageiro morreu).

A coluna boat também deve ser removida pois contém muitos valores nulos e pode ser considerada uma leaky feature (informa que o passageiro sobreviveu).


In [49]:
mask = df.isnull().any(axis = 1) # Criando uma máscara para selecionar as linhas com valores nulos
# any é um método que retorna True se qualquer elemento do eixo for True
mask.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [50]:
df[mask].body.head()

0      NaN
1      NaN
2      NaN
3    135.0
4      NaN
Name: body, dtype: float64

In [57]:
df.sex.value_counts() # Verificando a quantidade de valores por categoria

sex
male      843
female    466
Name: count, dtype: int64

In [55]:
df.embarked.value_counts(dropna=False) # Verificando a quantidade de valores por categoria, incluindo os valores nulos

embarked
S      914
C      270
Q      123
NaN      2
Name: count, dtype: int64

Temos 2 valores nulos na coluna embarked, podemos substituir pela moda (valor mais frequente), remover a linha, ou criar um dummy, ou então substituir pela média.